In [64]:
# Resolviendo el 8-puzzle con un algoritmo sin informacion (BFS)
import numpy as np
from abc import ABC, abstractmethod

from lib.eight_puzzle.problem import new_random_problem
from lib.eight_puzzle.board import new_board_from_state_serial, new_solved_board

In [65]:
# ======== CONSTANTS ========
BOARD_SIZE = 3

In [66]:
# ======== UTILIS ========
# Para saber donde esta el espacio vacio cada iteracion
class Coord_Empty:    
    def __init__(self, state):
        for i in range(state.shape[0]):
            for j in range(state.shape[1]):
                if state[i, j] == np.empty:
                    self.i = i
                    self.j = j
                    return

# Clases responsables de generar un nuevo estado dependiendo de la accion (modificar puzzle)
class Action(ABC):
    @abstractmethod
    def move(self, state):
        pass

class ActionUp(Action):
    name = "up"
    def move(self, state):
        # Revisar que sí se pueda mover hacia arriba
        coord_empty = Coord_Empty(state)
        if coord_empty.i == 0:
            return None
        else:  
            next_state = np.copy(state)
            buffer = next_state[coord_empty.i - 1, coord_empty.j]
            next_state[coord_empty.i, coord_empty.j] = buffer
            next_state[coord_empty.i - 1, coord_empty.j] = np.empty
            return next_state

class ActionDown(Action):
    name = "down"
    def move(self, state):
        # Revisar que sí se pueda mover hacia arriba
        coord_empty = Coord_Empty(state)
        if coord_empty.i == state.shape[0] - 1:
            return None
        else:  
            next_state = np.copy(state)
            buffer = next_state[coord_empty.i + 1, coord_empty.j]
            next_state[coord_empty.i, coord_empty.j] = buffer
            next_state[coord_empty.i + 1, coord_empty.j] = np.empty
            return next_state
        
class ActionLeft(Action):
    name = "left"
    def move(self, state):
        # Revisar que sí se pueda mover a la izquierda
        coord_empty = Coord_Empty(state)
        if coord_empty.j == 0:
            return None
        else:
            next_state = np.copy(state)
            buffer = next_state[coord_empty.i, coord_empty.j - 1]
            next_state[coord_empty.i, coord_empty.j] = buffer
            next_state[coord_empty.i, coord_empty.j - 1] = np.empty
            return next_state

class ActionRight(Action):
    name = "right"
    def move(self, state):
        # Revisar que sí se pueda mover a la derecha
        coord_empty = Coord_Empty(state)
        if coord_empty.j == state.shape[1] - 1:
            return None
        else:  
            next_state = np.copy(state)
            buffer = next_state[coord_empty.i, coord_empty.j + 1]
            next_state[coord_empty.i, coord_empty.j] = buffer
            next_state[coord_empty.i, coord_empty.j + 1] = np.empty
            return next_state

# Como se usa un grafo, cada nodo debe construir sobre las secuencias de movimientos necesarias para resolver el puzzle
class Node:
    def __init__(self, id, depth, state, parent, parentAction):
        self.id = id
        self.depth = depth
        self.state = state
        self.parent = parent
        self.parentAction = parentAction
        
# El algoritmo de BFS
class BFS:
    def __init__(self):
        self.actions = np.array([ActionUp(), ActionDown(), ActionLeft(), ActionRight()])
        self.goal = new_solved_board(BOARD_SIZE).to_np_matrix()
    
#   Para revisar si el estado generado es igual a nuestra meta
    def goalAchieved(self, state):
        return np.array_equal(state, self.goal)
    
#   La busqueda sin informacion
    def search(self, node):
        to_check = [node]
#       Para solo evaluar estados que no se hayan evaluado previamente
        checked = []
        curr_id = 0
        
        while to_check:
            curr_node = to_check.pop(0)
            
            if self.goalAchieved(curr_node.state):
                print("Puzzle ya estaba completado.")
                return curr_node
            
            for action in self.actions:
                next_state = action.move(curr_node.state)
#                 Considerar que el movimiento podria ser invalido
                if next_state is not None:
                    curr_id += 1
                    child = Node(curr_id, curr_node.depth + 1, next_state, curr_node, action)
                    
#                   Checar que no se haya revisado esta combinacion previamente
                    if child.state.tolist() not in checked:
                        to_check.append(child)
                        checked.append(child.state.tolist())
#                       Revisar si se encontro la solucion
                        if self.goalAchieved(child.state):
                            print("Puzzle completado.")
                            return child
        return None
        
    # Para imprimir la secuencia de movimientos de la respuesta final
    def print_solution(self, final_node):
    #   Reconstruir secuencia de movimientos usando la referencia a parent
        solution = [final_node]
        buffer = final_node.parent

        while buffer is not None:
            solution = np.insert(solution, 0, buffer)
            buffer = buffer.parent 

        for node in solution:
    #       Si llegamos a la raiz de la busqueda
            if(node.parent is None):
                print("INICIO")
                print("Estado")
                print(str(node.state) + "\n")
            else:
                print("Movimiento : " + str(node.parentAction.name))
                print("Profundidad : " + str(node.depth))
                print("Estado")
                print(str(node.state) + "\n")

In [67]:
# Creando 8-puzzle ejemplo

RANDOM_SEED = 23948752
MIN_ACTIONS = 100
MAX_ACTIONS = 1000

problem = new_random_problem(BOARD_SIZE, RANDOM_SEED, MIN_ACTIONS, MAX_ACTIONS)
board = new_board_from_state_serial(problem.initial_state().serialize())
puzzle = board.to_np_matrix()

puzzle

# La respuesta debe ser
#   1 2
# 3 4 5
# 6 7 8

matrix([[<built-in function empty>, 2, 7, 10],
        [1, 4, 12, 11],
        [6, 8, 15, 13],
        [5, 3, 9, 14]], dtype=object)

In [ ]:
# Necesitamos comenzar a construir el grafo, usando el puzzle declarado previamente como su estado inicial
root = Node(0, 0, puzzle, None, None)
bfs = BFS()

# Buscando solucion
solution = bfs.search(root)

if solution is None:
    print("No hay solucion para este 8-puzzle.")
else:
    bfs.print_solution(solution)